# Proyecto BI - Análitica de Texto según ODS

## Etapa 1

## Objetivos

*   Aplicar la metodología de analítica de textos para la construcción de soluciones de analítica alineadas con los objetivos del negocio en un contexto de aplicación.
*   Planear la interacción con un grupo interdisciplinario para identificar usuarios y posibles herramientas a desarrollar para la interacción del resultado del modelo desarrollado.

## Problema
<p style="text-align: justify;"> Desarrollar un modelo de clasificación, con técnicas de aprendizaje automático, que permita relacionar de manera automática un texto según los ODS. Al igual que desarrollar una aplicación que facilite la interacción con el resultado de dicho modelo. El modelo podrá ser utilizado entonces para la interpretación y análisis de la información textual que es recopilada a través de diferentes fuentes por UNFPA en procesos de planeación participativa para el desarrollo a nivel territorial.


### 1. Instalación e importanción de librerías.

In [37]:
!pip install ftfy

In [38]:
pip install openpyxl

In [39]:
!pip install unidecode

In [40]:
# Librería para manejar las contracciones que se presentan en el inglés.
!pip install contractions

In [41]:
# librería para manejar las flexiones gramaticales en el idioma inglés.
!pip install inflect
!pip install pandas-profiling==2.7.1

ERROR: Could not find a version that satisfies the requirement pandas-profiling==2.7.1 (from versions: 3.0.0, 3.1.0, 3.2.0, 3.3.0, 3.4.0, 3.5.0, 3.6.0, 3.6.1, 3.6.2, 3.6.3, 3.6.4, 3.6.5, 3.6.6)
ERROR: No matching distribution found for pandas-profiling==2.7.1


In [42]:
# librería Natural Language Toolkit, usada para trabajar con textos 
import nltk
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
# ¿Qué es el lema de una palabra? ¿Qué tan dificil puede ser obtenerlo, piensa en el caso en que tuvieras que escribir la función que realiza esta tarea?
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
# Instalación de librerias
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport
from unidecode import unidecode

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix
#from sklearn.metrics import plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin
from nltk.tokenize import word_tokenize
from sklearn.metrics import (
    ConfusionMatrixDisplay, RocCurveDisplay,
    roc_auc_score, precision_score, recall_score, f1_score
)
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt


In [45]:
import warnings
warnings.filterwarnings("ignore")

In [46]:
# Configuring pandas to show all cell content
pd.set_option("display.max_colwidth", None)

In [47]:
# Downloading stopwords
nltk.download("punkt")
nltk.download("stopwords")
stop_words = stopwords.words("spanish")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
stop_words


['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

### 2. Perfilamiento y entendimiento de los datos


#### Lectura de los datos.

In [49]:
# Uso de la libreria pandas para la lectura de archivos excel
df = pd.read_excel('./data/cat_345.xlsx')
# Asignación a una nueva variable de los datos leidos
data_t=df


In [50]:
data_t.shape

(3000, 2)

In [51]:
data_t.sample(5)

,Textos_espanol,sdg
2399,"También han llevado a cabo una variedad de proyectos de investigación relacionados con la condición de las mujeres y las niñas. Por ejemplo, la Comisión Nacional de Mujeres Libanesas inició una campaña para presionar por una reforma legal en el campo de los derechos económicos y publicó un estudio sobre los derechos de las mujeres en la ley libanesa. A menudo se cita a Marruecos por su compromiso con la presupuestación sensible al género. Publica un informe presupuestario anual que brinda todos los detalles sobre sus gastos desglosados ​​por sexo, y recientemente dio un paso más con una Ley Orgánica de Finanzas (2014) que consagra la igualdad de género como un principio clave en los objetivos e indicadores de desempeño del gobierno nacional marroquí. presupuesto.51 Además, Egipto ha creado una Unidad de Igualdad de Oportunidades dentro de su Ministerio de Finanzas, y Palestina ha formado un Comité Nacional para supervisar el desarrollo de presupuestos más sensibles al género en el futuro.",5
325,"En Ulaanbaatar, las pruebas de metales pesados ​​mostraron que las concentraciones de plomo, cadmio y mercurio en 58 muestras estaban dentro de las normas de Mongolia (MNS 5850:2008). Kosheleva, Natalia E. et al. La contaminación por metales pesados ​​proviene de diferentes fuentes, como la producción de curtidos, las actividades mineras, las zonas de tráfico y la ingeniería de calor y energía.",3
2297,"Las mujeres informaron el 32 % de las noticias danesas, pero los periodistas hombres y mujeres no informan por igual sobre todos los temas noticiosos. Las mujeres informaron solo el 15% de las noticias sobre política o economía y las reporteras también eran más propensas que los hombres a utilizar sujetos noticiosos femeninos y poner a una mujer en el centro de atención de una historia. Sin embargo, la mayoría de los periodistas hombres y mujeres utilizaron fuentes de noticias masculinas y solo el 13 % de las noticias tenían a una mujer en el centro de atención en 2015.",5
2669,"Las estrategias para el empoderamiento económico de las mujeres deben tener en cuenta las diferencias en las experiencias, necesidades y prioridades de las mujeres. Existen sólidos fundamentos de desarrollo para mejorar el acceso de las mujeres a una variedad de recursos económicos y financieros. Un conjunto de fundamentos gira en torno a las implicaciones del acceso de las mujeres a los recursos para el bienestar de sus hijos.",5
763,"En estos sistemas, la asistencia sanitaria la presta principalmente el sector público y se financia a través de los impuestos. Dos rasgos distintivos del sistema finlandés son un alto grado de descentralización y un sistema paralelo de financiación que combina la fiscalidad y el seguro nacional de enfermedad (Figura 2.4). Excluida la asistencia sanitaria ocupacional y estudiantil, así como la asistencia dental. Inversiones sanitarias brutas, asistencia sanitaria ocupacional y estudiantil y aparatos terapéuticos.",3


## 2.1 Limpieza de datos

In [52]:
data_t["sdg"].value_counts(dropna=False, normalize=True)

sdg
3    0.333333
4    0.333333
5    0.333333
Name: proportion, dtype: float64

In [53]:
data_t["Textos_espanol"] = data_t["Textos_espanol"].astype(str)

### Corrección de palabras mal codificadas

In [54]:
import ftfy

def fix_malformed_words(text):
    # Utiliza ftfy para corregir problemas de codificación
    text = ftfy.fix_text(text)
    return text

# Aplica la función a tu DataFrame
data_t['Textos_espanol'] = data_t['Textos_espanol'].apply(fix_malformed_words)



In [56]:
data_t.sample(5)

,Textos_espanol,sdg
725,"La atención a los resultados sanitarios comenzó en 1992 con la estrategia gubernamental ""Salud de la nación"" (Departamento de Sanidad, 1992a), que se basaba en el documento anterior de la OMS, ""Salud para todos en el año 2000"". Las enfermedades mentales se incluyeron como parte fundamental de esta estrategia, que establecía objetivos para reducir la morbilidad y la mortalidad debidas a las enfermedades mentales, y fue seguida de estrategias de aplicación en los Manuales del Área Clave de las Enfermedades Mentales (Department of Health, 1993, Department of Health, 1994, véase también Jenkins, 1994). En ""Building Bridges"" (Departamento de Sanidad, 1995) se expuso la visión del enlace intersectorial en torno a la salud mental y en ""The Spectrum of Care"" (Departamento de Sanidad, 1996) se estableció la gama de servicios e intervenciones previstos como parte de los servicios locales para enfermos mentales. El Quality Outcomes Framework (QoF) se introdujo en 2004 junto con el PbR, como un sistema de gestión del rendimiento y de pago de incentivos para los médicos generalistas (GP) en Inglaterra, e incluye indicadores de atención a la salud mental (véase la sección 5.2).",3
717,"Sin embargo, esta retroalimentación a los médicos y las campañas de información podrían estar mejor enfocadas, ya que los médicos y los pacientes son un objetivo más amplio. De hecho, en el caso del Reino Unido, las prácticas de retroalimentación centradas en los médicos con las tasas más altas de prescripción de antibióticos fueron eficaces para reducir su uso, mientras que las campañas de información centradas en el paciente tuvieron efectos limitados (OCDE, 2017b). Los hospitales representan alrededor de una cuarta parte de las ventas farmacéuticas totales en Francia, y la proporción del uso de genéricos allí fue solo del 2,3 % en valor en 2013 (ANSM, 2014). Esta baja proporción se explica en parte por las diferentes patologías de los pacientes y los tratamientos más intensivos e innovadores de los hospitales (OCDE, 2015b), pero los productores también suelen vender medicamentos de marca a precios reducidos para ganar un mercado más amplio (Dahan, 2016).",3
1546,"Pero en mis conversaciones con ministros de educación de todo el mundo, los desafíos que citan con más frecuencia no se refieren al diseño de reformas, sino a cómo se pueden poner en práctica con éxito. Las escuelas, colegios, universidades y otras instituciones educativas se encuentran entre los principales receptores del gasto público. Y como todos han participado en la educación, todos tienen una opinión al respecto. Todo el mundo apoya la reforma educativa, excepto cuando pueda afectar a sus propios hijos.",4
852,"Estos valiosos esfuerzos han incluido, en particular, información sobre los médicos por especialidad. En Finlandia, por ejemplo, la planificación de la mano de obra sanitaria forma parte de un ejercicio de planificación de la mano de obra para toda la economía, en lugar de ser específica para cada ocupación. El principal objetivo de la planificación global de la mano de obra es asesorar sobre la admisión de estudiantes de educación terciaria, para lograr un mejor equilibrio entre la oferta y la demanda futuras de mano de obra. Mientras tanto, Japón ha llevado a cabo un análisis de médicos, enfermeros, trabajadores de cuidados de larga duración, farmacéuticos y otros trabajadores sanitarios (Ono et al., Estos esfuerzos deben continuar y ampliarse para incluir a otras partes de la mano de obra, como los profesionales sanitarios aliados.",3
822,"Un incidente de gran importancia en la historia de la atención de la salud mental en Japón fue el ataque con cuchillo del entonces embajador de EE. UU. en Japón, Edwin Reischauer, por parte de una persona con esquizofrenia en 1964. Este ataque generó una alarma pública significativa y contribuyó a aumentar el estigma de las enfermedades mentales. , impulsado por una campaña en los medios de comunicación, además d